In [1]:
import torch
import torch.nn as nn
from torch import optim

import numpy as np
import pandas as pd

from utils import FaceKeypointDataset, transform, train_test_split, train
from model.network import Net

In [2]:
annotations_df = pd.read_csv("./data/AFLW/annotations.csv")#'data/Annotations/annotations.csv')
images_dir = '//OMVICECAVE/nas/ml_data/AFLW/annotaded_imgs/'#'data/Images/'

In [3]:
dataset = FaceKeypointDataset(annotations=annotations_df, 
                              root_dir=images_dir, 
                              transform=transform)

In [4]:
train_loader, val_loader, test_loader = train_test_split(dataset, 
                                                         train_size=0.8, 
                                                         val_size=0.1, 
                                                         batch_size=64)

In [5]:
model = Net()

In [6]:
criterion_keypoints = nn.MSELoss()
criterion_bbox = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [7]:
train(model, 
      criterion_keypoints, 
      criterion_bbox, 
      optimizer, 
      train_loader, 
      val_loader, 
      epochs=50, 
      device='cuda', 
      bbox_weight=1.0)

Epoch 1/50 | Train Loss: 157732.3793 | Val Loss: 143405.7422
Best model saved at epoch 1 with Val Loss: 143405.7422


Epoch 2/50 | Train Loss: 148942.3107 | Val Loss: 142651.7706
Best model saved at epoch 2 with Val Loss: 142651.7706


Epoch 3/50 | Train Loss: 147161.0543 | Val Loss: 145542.1689


Epoch 4/50 | Train Loss: 146131.1639 | Val Loss: 145430.8819


Epoch 5/50 | Train Loss: 145096.6185 | Val Loss: 204477.8606


...

Epoch 46/50 | Train Loss: 44727.9092 | Val Loss: 129228.6901


Epoch 47/50 | Train Loss: 43746.1577 | Val Loss: 123441.5325


Epoch 48/50 | Train Loss: 42779.9909 | Val Loss: 116921.7194
Best model saved at epoch 48 with Val Loss: 116921.7194


Epoch 49/50 | Train Loss: 39335.3378 | Val Loss: 119767.6866


Epoch 50/50 | Train Loss: 43800.6938 | Val Loss: 143254.4829
